<a href="https://colab.research.google.com/github/NitishaJonnada/nitisha_INFO5731_Spring2023/blob/main/In_class_exercise_05_04182023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

# New Section

In [2]:
# Write your code here

import pandas as pd
import nltk
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from textblob import TextBlob
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def reading_data(file_path):
  text_data, sentiments = [], []
  file_data = open(file_path).read()
  for i, j in enumerate(file_data.split("\n")):
    after_split = j.split(' ')
    text_data.append(" ".join(after_split[1:]))
    sentiments.append(after_split[0])
  return text_data, sentiments

In [6]:

training_text_data, training_sentiments = reading_data('stsa-train.txt')
training_df = pd.DataFrame(list(zip(training_sentiments, training_text_data)), columns = ['Sentimental Value', 'Raw Data'])
testing_text_data, testing_sentiments = reading_data('stsa-test.txt')
testing_df = pd.DataFrame(list(zip(testing_sentiments, testing_text_data)), columns = ['Sentimental Value', 'Raw Data'])

In [9]:
#Pre-processing
#removal of special characters
training_df['After noise removal'] = training_df['Raw Data'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
testing_df['After noise removal'] = testing_df['Raw Data'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
# removal of Punctuation
training_df['Punctuation removal'] = training_df['After noise removal'].str.replace('[^\w\s]','')
testing_df['Punctuation removal'] = testing_df['After noise removal'].str.replace('[^\w\s]','')
# Stopwords removal
stop_word = stopwords.words('english')
training_df['Stopwords removal'] = training_df['Punctuation removal'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_word))
testing_df['Stopwords removal'] = testing_df['Punctuation removal'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_word))
# Lower Casing
training_df['Lower casing'] = training_df['Stopwords removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))
testing_df['Lower casing'] = testing_df['Stopwords removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [10]:
tfidf_vector = TfidfVectorizer(analyzer = 'word')
tfidf_vector.fit(training_df['Lower casing'])
x =  tfidf_vector.transform(training_df['Lower casing'])
tfidf_vector_test = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vector.vocabulary_)
tfidf_vector_test.fit(testing_df['Lower casing'])
test_values_x = tfidf_vector_test.transform(testing_df['Lower casing'])


In [11]:
xtrain, xvalid, ytrain, yvalid = model_selection.train_test_split(x, training_df['Sentimental Value'],test_size=0.2)

In [12]:
def csv(model, x_data, y_data):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7, shuffle=True)
  return cross_val_score(model, x_data, y_data, cv=kfold).mean()

In [13]:
def training_model(model_intializer):
  model = model_intializer
  model.fit(xtrain, ytrain)
  predicted = model.predict(xvalid)
  accuracy = accuracy_score(predicted, yvalid)
  print("Accuracy of Traning data: {0}".format(accuracy))
  print(classification_report(yvalid, predicted))
  predicted_testing = model.predict(test_values_x)
  accuracy_testing = accuracy_score(predicted_testing, testing_df['Sentimental Value'])
  print("Accuracy of Testing data: {0}".format(accuracy_testing))
  print(classification_report(testing_df['Sentimental Value'], predicted_testing))
  if 'XGB' not in str(model):
    print("Cross validation score obtained: {0}".format(csv(model, test_values_x, testing_df['Sentimental Value'])))

In [14]:
training_model(naive_bayes.MultinomialNB())

Accuracy of Traning data: 0.8007220216606499
              precision    recall  f1-score   support

           0       0.84      0.73      0.78       671
           1       0.77      0.87      0.82       714

    accuracy                           0.80      1385
   macro avg       0.81      0.80      0.80      1385
weighted avg       0.80      0.80      0.80      1385

Accuracy of Testing data: 0.7903402854006586
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.86      0.70      0.77       912
           1       0.74      0.88      0.81       909

    accuracy                           0.79      1822
   macro avg       0.53      0.53      0.53      1822
weighted avg       0.80      0.79      0.79      1822

Cross validation score obtained: 0.7365309553834145


In [15]:
training_model(svm.SVC())

Accuracy of Traning data: 0.7935018050541516
              precision    recall  f1-score   support

           0       0.80      0.76      0.78       671
           1       0.78      0.83      0.80       714

    accuracy                           0.79      1385
   macro avg       0.79      0.79      0.79      1385
weighted avg       0.79      0.79      0.79      1385

Accuracy of Testing data: 0.7925356750823271
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.82      0.75      0.78       912
           1       0.77      0.83      0.80       909

    accuracy                           0.79      1822
   macro avg       0.53      0.53      0.53      1822
weighted avg       0.79      0.79      0.79      1822

Cross validation score obtained: 0.7217017954722873


In [16]:
training_model(KNeighborsClassifier(n_neighbors = 5))

Accuracy of Traning data: 0.5292418772563177
              precision    recall  f1-score   support

           0       0.81      0.04      0.07       671
           1       0.52      0.99      0.68       714

    accuracy                           0.53      1385
   macro avg       0.66      0.51      0.38      1385
weighted avg       0.66      0.53      0.39      1385

Accuracy of Testing data: 0.5104281009879253
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.71      0.04      0.07       912
           1       0.50      0.98      0.67       909

    accuracy                           0.51      1822
   macro avg       0.41      0.34      0.25      1822
weighted avg       0.61      0.51      0.37      1822

Cross validation score obtained: 0.5005464480874318


In [17]:
training_model(DecisionTreeClassifier())

Accuracy of Traning data: 0.6765342960288808
              precision    recall  f1-score   support

           0       0.65      0.71      0.68       671
           1       0.70      0.64      0.67       714

    accuracy                           0.68      1385
   macro avg       0.68      0.68      0.68      1385
weighted avg       0.68      0.68      0.68      1385

Accuracy of Testing data: 0.6580680570801317
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.65      0.67      0.66       912
           1       0.66      0.64      0.65       909

    accuracy                           0.66      1822
   macro avg       0.44      0.44      0.44      1822
weighted avg       0.66      0.66      0.66      1822

Cross validation score obtained: 0.5977121239416322


In [18]:
training_model(RandomForestClassifier())

Accuracy of Traning data: 0.7415162454873646
              precision    recall  f1-score   support

           0       0.72      0.76      0.74       671
           1       0.76      0.72      0.74       714

    accuracy                           0.74      1385
   macro avg       0.74      0.74      0.74      1385
weighted avg       0.74      0.74      0.74      1385

Accuracy of Testing data: 0.7447859495060373
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.74      0.76      0.75       912
           1       0.75      0.73      0.74       909

    accuracy                           0.74      1822
   macro avg       0.50      0.50      0.50      1822
weighted avg       0.74      0.74      0.74      1822

Cross validation score obtained: 0.6668528193118356


(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 

In [26]:
#Write your code here.
#import libraries
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data_df = pd.read_csv('Amazon_Unlocked_Mobile.csv')

In [ ]:
print("Shape: {0}".format(data_df.shape))

In [ ]:
cluster_data_df = data_df.head(1000)
cluster_data_df

In [ ]:
# removal of special characters
cluster_data_df['After noise removal'] = cluster_data_df['Reviews'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

#removal of punctations
cluster_data_df['Punctuation removal'] = cluster_data_df['After noise removal'].str.replace('[^\w\s]','')

# Removing numbers
cluster_data_df['Remove numbers'] = cluster_data_df['Punctuation removal'].str.replace('\d+', '')

#removal of stopwords
stop_word = stopwords.words('english')
cluster_data_df['Stopwords removal'] = cluster_data_df['Remove numbers'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_word))

# Lower Casing
cluster_data_df['Lower casing'] = cluster_data_df['Stopwords removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# Tokenization
cluster_data_df['Tokenization'] = cluster_data_df['Lower casing'].apply(lambda x: TextBlob(x).words)

# Stemming
st = PorterStemmer()
cluster_data_df['Stemming'] = cluster_data_df['Tokenization'].apply(lambda x: " ".join([st.stem(word) for word in x]))

# Lemmatization
cluster_data_df['Lemmatization'] = cluster_data_df['Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
cluster_data_df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(cluster_data_df['Lemmatization'].values)
tfidf.shape

In [ ]:
from sklearn.cluster import KMeans
model_tf = KMeans(n_clusters = 10, n_jobs = -1,random_state=99)
model_tf.fit(tfidf)

In [ ]:
labels_tf = model_tf.labels_
cluster_center_tf=model_tf.cluster_centers_
cluster_center_tf

In [ ]:
terms1 = tfidf_vect.get_feature_names()

In [ ]:
terms1[1:10]

In [ ]:
silhouette_score_tf = metrics.silhouette_score(tfidf, labels_tf, metric='euclidean')
silhouette_score_tf

In [ ]:
df1 = cluster_data_df
df1['Tfidf Clus Label'] = model_tf.labels_
df1.head(5)

In [ ]:
df1.groupby(['Tfidf Clus Label'])['Reviews'].count()

In [ ]:
print("Top terms per cluster:")
order_centroids = model_tf.cluster_centers_.argsort()[:, ::-1]
for i in range(10):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms1[ind], end='')
        print()

In [ ]:
import matplotlib.pyplot as plt
plt.bar([x for x in range(10)], df1.groupby(['Tfidf Clus Label'])['Reviews'].count(), alpha = 0.4)
plt.title('KMeans cluster points')
plt.xlabel("Cluster number")
plt.ylabel("Number of points")
plt.show()

In [ ]:
for i in range(10):
    print("4 review of assigned to cluster ", i)
    print("-" * 70)
    print(df1.iloc[df1.groupby(['Tfidf Clus Label']).groups[i][0]]['Reviews'])
    print('\n')
    print(df1.iloc[df1.groupby(['Tfidf Clus Label']).groups[i][5]]['Reviews'])
    print('\n')
    print(df1.iloc[df1.groupby(['Tfidf Clus Label']).groups[i][10]]['Reviews'])
    print('\n')
    print("_" * 70)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
bow = count_vect.fit_transform(cluster_data_df['Reviews'].values)
bow.shape

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters = 10,init='k-means++', n_jobs = -1,random_state=99)
model.fit(bow)

In [ ]:
labels = model.labels_
cluster_center=model.cluster_centers_

In [ ]:
from sklearn import metrics
print(metrics.silhouette_score(bow, labels, metric='euclidean'))

In [ ]:
cluster_data_df['Bow Clus Label'] = model.labels_ # the last column you can see the label numebers
cluster_data_df.head(2)

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

In [ ]:
# Computing 200th Nearest neighbour distance
minPts = 2 * 100
# Lower bound function copied from -> https://gist.github.com/m00nlight/0f9306b4d4e61ba0195f
def lower_bound(nums, target): # This function return the number in the array just greater than or equal to itself.
    l, r = 0, len(nums) - 1
    while l <= r: # Binary searching.
        mid = int(l + (r - l) / 2)
        if nums[mid] >= target:
            r = mid - 1
        else:
            l = mid + 1
    return l

def compute200thnearestneighbour(x, data): # Returns the distance of 200th nearest neighbour.
    dists = []
    for val in data:
        dist = np.sum((x - val) **2 ) # computing distances.
        if (len(dists) == 200 and dists[199] > dist): # If distance is larger than current largest distance found.
          l = int(lower_bound(dists, dist)) # Using the lower bound function to get the right position.
          if l < 200 and l >= 0 and dists[l] > dist:
              dists[l] = dist
        else:
          dists.append(dist)
          dists.sort()
    
    return dists[199] # Dist 199 contains the distance of 200th nearest neighbour.

In [ ]:
list_of_sent_train = list()

for i in cluster_data_df["Lower casing"].values:
  list_of_sent_train.append(i.split())

In [ ]:
import gensim
w2v_model=gensim.models.Word2Vec(list_of_sent_train, size=100, workers=4)

In [ ]:
import numpy as np
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this train
count = 1
for sent in list_of_sent_train: # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    cnt_words =1; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)

In [ ]:
twohundrethneigh = []
for val in sent_vectors[:300]:
    twohundrethneigh.append(compute200thnearestneighbour(val, sent_vectors[:300]) )
twohundrethneigh.sort()

In [ ]:
model = DBSCAN(eps = 5, min_samples = minPts, n_jobs=-1)
model.fit(sent_vectors)

In [ ]:
cluster_data_df['AVG-W2V Clus Label'] = model.labels_
cluster_data_df.head(2)

In [ ]:
import scipy
from scipy.cluster import hierarchy
dendro = hierarchy.dendrogram(hierarchy.linkage(sent_vectors,method = 'ward'))
plt.axhline(y = 10) # cut at 30 to get 5 clusters

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')  #took n=5 from dendrogram curve 
Agg=cluster.fit_predict(sent_vectors)

In [ ]:
#Giving Labels/assigning a cluster to each point/text 
aggdfa = cluster_data_df
aggdfa['AVG-W2V Clus Label'] = cluster.labels_
aggdfa.head(2)

In [ ]:
aggdfa.groupby(['AVG-W2V Clus Label'])['Reviews'].count()

In [ ]:
for i in range(5):
    print("2 reviews of assigned to cluster ", i)
    print("-" * 70)
    print(aggdfa.iloc[aggdfa.groupby(['AVG-W2V Clus Label']).groups[i][0]]['Reviews'])
    print('\n')
    print(aggdfa.iloc[aggdfa.groupby(['AVG-W2V Clus Label']).groups[i][1]]['Reviews'])
    print('\n')
    print("_" * 70)

In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

In [ ]:
#You can write you answer here. (No code needed)

k-means splis a large dataset into a fixed number of clustes and generalises them into different sizes and shape. It doesnt work much efficiently with noisy data but it works efficiently with large datasets. 
DBscan is used to sepeate clusters that has high density from the clusters with low density. It works efficiently for noisy data but does not work efficiently with large datasets. 
Hierarchial Clustering is used to group objects that are similar into groups that are called as clusters. It is challenging to classify the clusters using Hierarchical c
